# Caso de uso: traducción de un texto en bloque
<div class="alert alert-warning">
    
<i class="fa fa-bug"></i>
Si no queréis perder los resultados de ejemplo, **no ejecutéis** el cuaderno de jupyter sin tener una clave propia de la API de OpenAI. Estos cuadernos solo cumplen una función ilustrativa de los códigos y formas de utilizar el modelo GPT-3 con Python. En caso de disponer de una **clave**, guardadla en un archivo **.env** para mayor seguridad como **un texto entrecomillado asignado a la variable OPENAI_API_KEY**.
</div>

## Autentificación

La API de OpenAI utiliza claves de API para la autentificación. 

<div class="alert alert-danger">
    
<i class="fa fa-exclamation-circle"></i>
Recuerda que tu clave de API es un secreto. No la compartas con otros ni la expongas en ningún código del lado del cliente (navegadores, aplicaciones). Las solicitudes de producción deben dirigirse a través de su propio servidor *backend*, donde su clave de API puede cargarse de forma segura desde una variable de entorno o un servicio de gestión de claves.

</div>

Todas las solicitudes de API deben incluir su clave de API, es importante almacenar en un documento seguro la llave. Para ello, crea un archivo nuevo `.env` para almacenarla en su interior de la siguiente forma:

`OPENAI_API_KEY = "MI_API_KEY"`

Con esto, para recuperar la clave de la API, tendremos que usar el **getenv** de `os`.

In [1]:
import os
import openai
from dotenv import load_dotenv
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

## Introducción

Como expuse al final del cuaderno 3.1, el problema de gestionar cada párrafo por separado, supone un problema para la coherencia y continuidad del texto para la traducción; lo ideal sería pasarle todo el conjunto de texto de una vez, que lo gestione y traduzca el bloque.

El problema que se presenta en este caso es que, al hacerlo, perdemos las referencias de los párrafos. Normalmente, cuando traducimos oraciones y frases, estas pueden variar en número de palabras, pero el número de frases y oraciones de entrada y salida es raro que varíen; por esta razón, podríamos generar un diccionario que contenga, por cada llave (número de párrafo) el número de puntos (frases) que contiene.

Ese diccionario de referencia nos puede servir para recuperar los párrafos en la traducción antes de guardar el documento.

**¿Por qué elegir esta opción y no añadir *\n* como unión de la lista de párrafos?** Aunque es una opción razonable y sencilla, debemos tener en cuenta que estamos trabajando con un modelo que presenta mucha flexibilidad a la hora de dar una respuesta, lo que podría significar que se *pierda* o decida eliminar alguno de los saltos de línea en el proceso. Por esta misma razón, al trabajar con un modelo en bruto, es siempre recomendable que los elementos que genere de peticiones que requieran cierto grado de equivalencia (como una traducción) sean revisados con cuidado.

In [ ]:
# Instalar biblioteca si no la tenéis
#!pip3 install python-docx

In [5]:
from docx import Document

# ----------------------------------------------- CARGA DEL DOCUMENTO CON docx.Document
documento = Document(docx='./documentos/para_traducir.docx').paragraphs

# ----------------------------------------------- UNIMOS TODOS LOS PÁRRAFOS EN UNA ÚNICA LISTA
doc = []
for parrafos in documento:
    doc.append(parrafos.text)
# ----------------------------------------------- CREAMOS EL DICCIONARIO DE REFERENCIA PARA LOS PÁRRAFOS
doc_split = {}
for val in range(len(doc)):
    doc_split[val] = len(doc[val].split(sep='.')) - 1

# ----------------------------------------------- PREPARAMOS EL BLOQUE DE TEXTO PARA GPT-3
doc = " ".join(doc)

print(doc)
print(doc_split)

En un lugar de la Mancha, de cuyo nombre no quiero acordarme, no ha mucho tiempo que vivía un hidalgo de los de lanza en astillero, adarga antigua, rocín flaco y galgo corredor. Una olla de algo más vaca que carnero, salpicón las más noches, duelos y quebrantos los sábados, lantejas los viernes, algún palomino de añadidura los domingos, consumían las tres partes de su hacienda. El resto della concluían sayo de velarte, calzas de velludo para las fiestas, con sus pantuflos de lo mesmo, y los días de entresemana se honraba con su vellorí de lo más fino. Tenía en su casa una ama que pasaba de los cuarenta y una sobrina que no llegaba a los veinte, y un mozo de campo y plaza que así ensillaba el rocín como tomaba la podadera. Frisaba la edad de nuestro hidalgo con los cincuenta años. Era de complexión recia, seco de carnes, enjuto de rostro, gran madrugador y amigo de la caza. Quieren decir que tenía el sobrenombre de «Quijada», o «Quesada», que en esto hay alguna diferencia en los autores

Una vez tenemos el bloque de texto preparado, hacemos una única llamada a GPT-3 para pedirle que traduzca, a un idioma determinado, el texto que tenemos.

Algo que debemos tener en cuenta es que GPT-3, a la hora de generar la respuesta, tal como construimos la petición, lo primero que generará es el espacio en blanco que hay detrás de los dos puntos de la solicitud; podemos evitar que aparezca este guardando directamente en la variable donde almacenamos la respuesta cogiendo desde el primer caracter y no toda la respuesta.

**¿Por qué aparece este espacio?** A la hora de interpretar un resultado, OpenAi recomienda que no existan espacios en blanco después del texto de solicitud ni tampoco saltos de línea.

In [6]:
idioma = 'inglés'

# ----------------------------------------------- TRADUCCIÓN CON GPT-3
entrada = 'Original: ' + doc + '\nTraducido al'+ idioma + ':'
traduccion = openai.Completion.create(engine='davinci',
                                      prompt=entrada,
                                      temperature=0.81,
                                      stop=['\n', 'Traducido al'+ idioma + ':'],
                                      max_tokens = int(len(doc)))
doc_traducido = traduccion['choices'][0]['text'][1:]
doc_traducido

'In a village of La Mancha, name I don\'t want to remember, lately a gentleman of the kind of lance in the sheath, old shield, the lean horse and the hound runner lived. The cattle on the most nights, the mourning and the take away on the Saturdays, the boiled lentils on Fridays, some diners on the Sundays, and the rest of the stuff with the gabardine ones, the fine pants for the holidays and the Saturdays with the fine gabardine, and the days of the week with the slippers of the same stuff. In his house lived an old woman over the forty and the niece of the twenty ones. And a farmhand and a worker who saddled the runner as well as he took the scythe. The age of the gentleman had about fifty years. He was of robust complexion, thick skin, lean face, and he was the man of the early riser and the hunt. They say that he had the nickname of "Quijada" or "Quesada", but in this there is some difference of the authors who wrote on this subject, although it should be understood that he called 

Esta respuesta de GPT-3 que tenemos almacenada en **doc_traducido** debemos reconstruirla para que vuelva a añadir los párrafos tal como debería aparecer; para lo que aplicaremos el diccionario que generamos al inicio **doc_split**.

Una vez hemos hecho eso, lo único que nos falta es guardar el archivo y revisar que lo haya hecho correctamente.

In [7]:
# ----------------------------------------------- REGENERAR LOS PÁRRAFOS
# ------------------------ Separamos todas las frases y le volvemos a añadir el punto a cada frase
doc_trad_split = doc_traducido.split(sep='.')
#puntos = [str(frase) + '.' for frase in doc_trad_split]
#puntos.remove('.')

# ------------------------ Utilizamos doc_split para volver a generar los párrafos correctamente
parrafos = []
for n in range(len(doc_split)):
    parrafos.append(doc_trad_split[:doc_split[n]])
    doc_trad_split = doc_trad_split[doc_split[n]:]

# ------------------------ Unimos cada párrafo que está sepada por frases en listas dentro de la lista del documento
doc_final = ['.'.join(parrafo) for parrafo in parrafos]
doc_final = [str(parrafo.lstrip()) + '.' for parrafo in doc_final]
print(doc_final)

# ----------------------------------------------- GUARDADO DEL DOCUMENTO
doc_TRAD = Document()
for parrafo in doc_final:
    doc_TRAD.add_paragraph(parrafo)

doc_TRAD.save('./documentos/traduccion_'+idioma+'.docx')

["In a village of La Mancha, name I don't want to remember, lately a gentleman of the kind of lance in the sheath, old shield, the lean horse and the hound runner lived. The cattle on the most nights, the mourning and the take away on the Saturdays, the boiled lentils on Fridays, some diners on the Sundays, and the rest of the stuff with the gabardine ones, the fine pants for the holidays and the Saturdays with the fine gabardine, and the days of the week with the slippers of the same stuff. In his house lived an old woman over the forty and the niece of the twenty ones.", 'And a farmhand and a worker who saddled the runner as well as he took the scythe. The age of the gentleman had about fifty years. He was of robust complexion, thick skin, lean face, and he was the man of the early riser and the hunt. They say that he had the nickname of "Quijada" or "Quesada", but in this there is some difference of the authors who wrote on this subject, although it should be understood that he call

### Conclusiones

Para poder usar correctamente GPT-3 es necesario entender su funcionamiento y, sobre todo, tener en cuenta que a veces, si el motoro o la temperatura no es la adecuada, no respondera como esperamos. Es necesario, por tanto, que revisemos los resultados que da. Sin duda, los bloques de texto a GPT-3 le sientan mejor que ir frase por frase; eso no significa que, al encontrarse caracteres especiales, pueda decidir *irse por las ramas*.

Hay que tener en cuenta, además, que OpenAi limita el número de palabras máximo que el modelo genera de una vez, habría que valorar según el caso si utilizar el código para hacer estas traducciones tal como aparece en el cuaderno 2.1 (y hacer la traducción por párrafos), o generar un nuevo planteamiento del problema en donde este trabajo sea reiterativo sobre el texto, donde las respuestas generadas vayan incluyéndose en la siguiente iteración para que GPT-3 interprete todo el conjunto.

<div class="alert alert-info">
    
<i class="fa fa-code"></i> **Este cuaderno ha sido creado con la ayuda de GPT-3.**
<hr/>
    
**Si tienes alguna duda relacionada con estos cuadernos, puedes contactar conmigo:**
Mª Reyes R.P. (Erebyel). **[Web](https://www.erebyel.es/) • [Twitter](https://twitter.com/erebyel) • [Linkedin](https://www.linkedin.com/in/erebyel/)**.
    
<hr/>
    
<i class="fa fa-plus-circle"></i> **Fuentes:**
* ***Documentación de la Beta de OpenAI***: https://beta.openai.com/docs/introduction
</div>